In [1]:
import pandas as pd
import numpy as np
import joblib
import os
import sys

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Cấu hình đường dẫn đầu vào
base_path = os.getcwd()
models_folder = os.path.join(base_path, 'models_and_results')
data_folder = os.path.join(base_path, 'folder_standardized')

print("--- KHỞI TẠO PREDICTION ---")

--- KHỞI TẠO PREDICTION ---


In [3]:
def load_components():
    """
    Tải các thành phần cần thiết cho dự báo:
    1. Scaler: MinMaxScaler đã fit trên tập Train
    2. Train columns: Danh sách tên cột để đảm bảo khớp với tập Train
    3. Models: 3 mô hình đã huấn luyện (LogisticRegression, RandomForest, XGBoost)
    
    Returns:
        tuple: (scaler, train_columns, model_lr, model_rf, model_gb)
        
    Raises:
        FileNotFoundError: Nếu thiếu bất kỳ file nào
    """
    print("\n1. LOAD COMPONENTS FOR PREDICTION")
    
    try:
        # Tải Scaler đã fit
        scaler = joblib.load(os.path.join(data_folder, 'minmax_scaler.pkl'))
        print("   - Đã tải Scaler (MinMaxScaler)")
        
        # Tải danh sách cột từ tập Train
        train_columns = pd.read_csv(os.path.join(data_folder, 'X_train.csv'), nrows=0).columns
        print("   - Đã tải danh sách cột Train")
        
        # Tải 3 mô hình
        print("   ℹ Đang tải 3 mô hình huấn luyện...")
        model_lr = joblib.load(os.path.join(models_folder, 'LogisticRegression.pkl'))
        print("   - Đã tải: LogisticRegression")
        
        model_rf = joblib.load(os.path.join(models_folder, 'RandomForest.pkl'))
        print("   - Đã tải: RandomForest")
        
        model_gb = joblib.load(os.path.join(models_folder, 'XGBoost.pkl'))
        print("   - Đã tải: XGBoost")
        
        print("   - Tất cả components đã tải thành công!")
        return scaler, train_columns, model_lr, model_rf, model_gb
        
    except FileNotFoundError as e:
        print(f"   - Lỗi: Thiếu file - {e}")
        print("      → Hãy chạy các file Training (EDA, processing, logisticRegression, randomForest, XGBoosting) trước!")
        sys.exit()

scaler, train_columns, model_lr, model_rf, model_gb = load_components()


1. LOAD COMPONENTS FOR PREDICTION
   - Đã tải Scaler (MinMaxScaler)
   - Đã tải danh sách cột Train
   ℹ Đang tải 3 mô hình huấn luyện...
   - Đã tải: LogisticRegression
   - Đã tải: RandomForest
   - Đã tải: XGBoost
   - Tất cả components đã tải thành công!


In [4]:
def preprocess_new_input(input_dict, scaler, train_columns):
    """
    Biến đổi dữ liệu nhập vào thành định dạng y hệt với dữ liệu Training.
    
    Quy trình:
    1. Tái tạo Feature Engineering (Has_Family, Num_Services, Payment_Type, Avg_Charges_Per_Service)
    2. Xử lý kiểu dữ liệu số và mã hóa (One-Hot Encoding)
    3. Sắp xếp lại cột để khớp với tập Train
    4. Chuẩn hóa (Scaling) các đặc trưng số
    
    Args:
        input_dict (dict): Dữ liệu nhập vào (các đặc trưng của khách hàng)
        scaler (MinMaxScaler): Scaler đã fit trên tập Train
        train_columns (Index): Danh sách tên cột từ tập Train
    
    Returns:
        DataFrame: Dữ liệu đã xử lý, sẵn sàng cho dự báo
    """
    df_new = pd.DataFrame([input_dict])
    
    # ====== BƯỚC 1: TÁI TẠO FEATURE ENGINEERING ======
    # Phải giống hoàn toàn với processing.ipynb để đảm bảo consistency
    
    # Feature 1: Has_Family - Khách hàng có gia đình (Partner hoặc Dependents)
    df_new['Has_Family'] = ((df_new['Partner'] == 'Yes') | (df_new['Dependents'] == 'Yes')).astype(int)
    
    # Feature 2: Num_Services - Tổng số dịch vụ gia tăng
    services = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
                'TechSupport', 'StreamingTV', 'StreamingMovies']
    # Đảm bảo các cột dịch vụ tồn tại (mặc định = 'No' nếu thiếu)
    for s in services:
        if s not in df_new.columns:
            df_new[s] = 'No'
    df_new['Num_Services'] = (df_new[services] == 'Yes').sum(axis=1)
    
    # Feature 3: Payment_Type - Phân loại hình thức thanh toán (Automatic vs Manual)
    if 'PaymentMethod' in df_new.columns:
        df_new['Payment_Type'] = df_new['PaymentMethod'].apply(
            lambda x: 'Automatic' if 'automatic' in str(x).lower() else 'Manual'
        )
    else:
        df_new['Payment_Type'] = 'Manual'  # Giá trị mặc định nếu thiếu
    
    # Feature 4: Avg_Charges_Per_Service - Cước phí trung bình trên mỗi dịch vụ
    df_new['Avg_Charges_Per_Service'] = df_new['MonthlyCharges'] / (df_new['Num_Services'] + 1)
    
    # ====== BƯỚC 2: XỬ LÝ KIỂU DỮ LIỆU VÀ ENCODING ======
    
    # Chuyển đổi TotalCharges sang kiểu số (mặc định 0 nếu NaN)
    df_new['TotalCharges'] = pd.to_numeric(df_new['TotalCharges'], errors='coerce').fillna(0)
    
    # One-Hot Encoding cho các biến phân loại (object)
    df_new = pd.get_dummies(df_new)
    
    # ====== BƯỚC 3: REINDEX ĐỂ KHỚP VỚI TẬP TRAIN ======
    # Thêm cột bị thiếu (với giá trị 0) và loại bỏ cột thừa
    df_new = df_new.reindex(columns=train_columns, fill_value=0)
    
    # ====== BƯỚC 4: SCALING ======
    cols_to_scale = ['tenure', 'MonthlyCharges', 'TotalCharges', 
                     'Num_Services', 'Avg_Charges_Per_Service']
    
    # Lấy tên các đặc trưng từ scaler
    if hasattr(scaler, 'feature_names_in_'):
        cols_in_scaler = scaler.feature_names_in_
        # Chỉ scale các cột thực sự tồn tại trong scaler
        final_cols_to_scale = [c for c in cols_to_scale if c in cols_in_scaler]
        
        if len(final_cols_to_scale) > 0:
            df_new[final_cols_to_scale] = scaler.transform(df_new[final_cols_to_scale])
    else:
        # Fallback cho phiên bản sklearn cũ (không có feature_names_in_)
        df_new[cols_to_scale] = scaler.transform(df_new[cols_to_scale])
    
    return df_new

In [5]:
def predict(input_data, scaler, train_columns, model_lr, model_rf, model_gb):
    """
    Thực hiện dự báo khách hàng có nguy cơ rời bỏ dịch vụ (Churn) hay không.
    
    Quy trình:
    1. Tiền xử lý dữ liệu nhập vào (Feature Engineering, Encoding, Scaling)
    2. Dự báo với 3 mô hình và ngưỡng tối ưu riêng
    3. Hiển thị kết quả so sánh 3 mô hình dưới dạng bảng rõ ràng
    
    Args:
        input_data (dict): Dữ liệu khách hàng cần dự báo
        scaler (MinMaxScaler): Scaler đã fit
        train_columns (Index): Danh sách cột từ tập Train
        model_lr (Pipeline): Mô hình Logistic Regression
        model_rf (Pipeline): Mô hình Random Forest
        model_gb (Pipeline): Mô hình XGBoost
    """
    # Tiền xử lý dữ liệu
    processed_data = preprocess_new_input(input_data, scaler, train_columns)
    
    # Cấu hình ngưỡng tối ưu cho mỗi mô hình
    # Những giá trị này được xác định từ quá trình training (threshold tuning)
    THRESHOLDS = {
        'LogisticRegression': 0.4686,
        'RandomForest': 0.6256,
        'XGBoost': 0.4500
    }
    
    # Hiển thị tiêu đề kết quả
    print("\n" + "="*90)
    print("KẾT QUẢ DỰ BÁO KHÁCH HÀNG - CHURN PREDICTION")
    print("="*90)
    
    # Thông tin khách hàng
    contract = input_data.get('Contract', 'N/A')
    internet = input_data.get('InternetService', 'N/A')
    monthly_charge = input_data.get('MonthlyCharges', 0)
    tenure = input_data.get('tenure', 0)
    
    print(f"Hợp đồng: {contract} | Internet: {internet} | Cước: ${monthly_charge:.2f} | Thời gian: {tenure} tháng")
    print("-" * 90)
    print(f"{'MÔ HÌNH':<22} | {'XÁC SUẤT':<12} | {'NGƯỠNG':<10} | {'KẾT LUẬN':<20}")
    print("-" * 90)
    
    # ====== LOGISTIC REGRESSION ======
    prob_lr = model_lr.predict_proba(processed_data)[0][1]
    threshold_lr = THRESHOLDS['LogisticRegression']
    res_lr = "- RỜI BỎ (Rủi ro)" if prob_lr >= threshold_lr else "- An toàn"
    print(f"{'Logistic Regression':<22} | {prob_lr:>6.2%}      | {threshold_lr:.4f}   | {res_lr:<20}")
    
    # ====== RANDOM FOREST ======
    prob_rf = model_rf.predict_proba(processed_data)[0][1]
    threshold_rf = THRESHOLDS['RandomForest']
    res_rf = "- RỜI BỎ (Rủi ro)" if prob_rf >= threshold_rf else "- An toàn"
    print(f"{'Random Forest':<22} | {prob_rf:>6.2%}      | {threshold_rf:.4f}   | {res_rf:<20}")
    
    # ====== XGBOOST ======
    prob_gb = model_gb.predict_proba(processed_data)[0][1]
    threshold_gb = THRESHOLDS['XGBoost']
    res_gb = "- RỜI BỎ (Rủi ro)" if prob_gb >= threshold_gb else "- An toàn"
    print(f"{'XGBoost':<22} | {prob_gb:>6.2%}      | {threshold_gb:.4f}   | {res_gb:<20}")
    
    print("="*90 + "\n")

In [6]:
def run_demo_predictions():
    """
    Chạy hai test case: một khách hàng rủi cao (case_risk) và một khách hàng an toàn (case_safe).
    Giúp minh họa khả năng dự báo của hệ thống.
    """
    print("\n" + "="*90)
    print("DEMO PREDICTIONS - CHẠY 2 TEST CASE")
    print("="*90)
    
    # ====== TEST CASE 1: KHÁCH HÀNG RỦI CAO ======
    # Đặc điểm: Mới ký hợp đồng tháng (tenure=1), hợp đồng ngắn hạn, Internet sợi quang đắt tiền
    print("\nTEST CASE 1: Khách hàng RỦI CAO (New Customer with High Risk)")
    case_risk = {
        'gender': 'Male',
        'SeniorCitizen': 0,
        'Partner': 'No',
        'Dependents': 'No',
        'tenure': 1,                          # ⚠ Rất mới: 1 tháng
        'PhoneService': 'Yes',
        'MultipleLines': 'No',
        'InternetService': 'Fiber optic',    # ⚠ Dịch vụ đắt nhất
        'OnlineSecurity': 'No',
        'OnlineBackup': 'No',
        'DeviceProtection': 'No',
        'TechSupport': 'No',
        'StreamingTV': 'No',
        'StreamingMovies': 'No',
        'Contract': 'Month-to-month',        # ⚠ Hợp đồng dễ hủy nhất
        'PaperlessBilling': 'Yes',
        'PaymentMethod': 'Electronic check',  # ⚠ Hình thức thanh toán không tự động
        'MonthlyCharges': 85.0,
        'TotalCharges': 85.0
    }
    predict(case_risk, scaler, train_columns, model_lr, model_rf, model_gb)
    
    # ====== TEST CASE 2: KHÁCH HÀNG AN TOÀN ======
    # Đặc điểm: Khách hàng lâu năm (48 tháng), hợp đồng dài hạn, có gia đình, dùng nhiều dịch vụ
    print("TEST CASE 2: Khách hàng AN TOÀN (Loyal Customer with Low Risk)")
    case_safe = {
        'gender': 'Female',
        'SeniorCitizen': 0,
        'Partner': 'Yes',                    # ✓ Có bạn đời
        'Dependents': 'Yes',                 # ✓ Có người phụ thuộc
        'tenure': 48,                        # ✓ Khách lâu năm: 4 năm
        'PhoneService': 'Yes',
        'MultipleLines': 'Yes',
        'InternetService': 'DSL',            # ✓ Dịch vụ ổn định
        'OnlineSecurity': 'Yes',
        'OnlineBackup': 'Yes',
        'DeviceProtection': 'Yes',
        'TechSupport': 'Yes',
        'StreamingTV': 'Yes',
        'StreamingMovies': 'Yes',
        'Contract': 'Two year',              # ✓ Hợp đồng dài hạn 2 năm
        'PaperlessBilling': 'No',
        'PaymentMethod': 'Bank transfer (automatic)',  # ✓ Thanh toán tự động
        'MonthlyCharges': 60.0,
        'TotalCharges': 3000.0
    }
    predict(case_safe, scaler, train_columns, model_lr, model_rf, model_gb)

# Chạy demo
run_demo_predictions()
print("\n--- HOÀN THÀNH PREDICTION ---")


DEMO PREDICTIONS - CHẠY 2 TEST CASE

TEST CASE 1: Khách hàng RỦI CAO (New Customer with High Risk)

KẾT QUẢ DỰ BÁO KHÁCH HÀNG - CHURN PREDICTION
Hợp đồng: Month-to-month | Internet: Fiber optic | Cước: $85.00 | Thời gian: 1 tháng
------------------------------------------------------------------------------------------
MÔ HÌNH                | XÁC SUẤT     | NGƯỠNG     | KẾT LUẬN            
------------------------------------------------------------------------------------------
Logistic Regression    | 39.47%      | 0.4686   | - An toàn           
Random Forest          | 89.00%      | 0.6256   | - RỜI BỎ (Rủi ro)   
XGBoost                | 88.39%      | 0.4500   | - RỜI BỎ (Rủi ro)   

TEST CASE 2: Khách hàng AN TOÀN (Loyal Customer with Low Risk)

KẾT QUẢ DỰ BÁO KHÁCH HÀNG - CHURN PREDICTION
Hợp đồng: Two year | Internet: DSL | Cước: $60.00 | Thời gian: 48 tháng
------------------------------------------------------------------------------------------
MÔ HÌNH                | XÁ

In [7]:
# ====== DÙNG CUSTOM INPUT ======
# Có thể tạo bộ dữ liệu khách hàng tùy chỉnh để dự báo
# Chỉ cần cung cấp tất cả các đặc trưng giống như trong dict dưới đây

print("\n" + "="*90)
print("CUSTOM PREDICTION - NHẬP DỮ LIỆU TÙY CHỈNH")
print("="*90)

my_customer = {
    'gender': 'Male',
    'SeniorCitizen': 0,
    'Partner': 'No',
    'Dependents': 'No',
    'tenure': 6,                            # Khách hàng 6 tháng
    'PhoneService': 'Yes',
    'MultipleLines': 'No',
    'InternetService': 'DSL',
    'OnlineSecurity': 'Yes',
    'OnlineBackup': 'Yes',
    'DeviceProtection': 'No',
    'TechSupport': 'No',
    'StreamingTV': 'No',
    'StreamingMovies': 'No',
    'Contract': 'One year',                 # Hợp đồng 1 năm
    'PaperlessBilling': 'Yes',
    'PaymentMethod': 'Credit card (automatic)',
    'MonthlyCharges': 55.0,
    'TotalCharges': 330.0
}

print("\nCUSTOM: Khách hàng với đặc điểm tùy chỉnh")
predict(my_customer, scaler, train_columns, model_lr, model_rf, model_gb)


CUSTOM PREDICTION - NHẬP DỮ LIỆU TÙY CHỈNH

CUSTOM: Khách hàng với đặc điểm tùy chỉnh

KẾT QUẢ DỰ BÁO KHÁCH HÀNG - CHURN PREDICTION
Hợp đồng: One year | Internet: DSL | Cước: $55.00 | Thời gian: 6 tháng
------------------------------------------------------------------------------------------
MÔ HÌNH                | XÁC SUẤT     | NGƯỠNG     | KẾT LUẬN            
------------------------------------------------------------------------------------------
Logistic Regression    | 23.49%      | 0.4686   | - An toàn           
Random Forest          | 43.17%      | 0.6256   | - An toàn           
XGBoost                | 31.39%      | 0.4500   | - An toàn           

